In [1]:
# import all the modules

import os
import csv
import requests
from zipfile import ZipFile
from io import BytesIO, TextIOWrapper
from urllib.request import urlopen
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pylab as plt


In [ ]:
# grab zip file MHSA dataset
response = requests.get('https://arlweb.msha.gov/OpenGovernmentData/DataSets/Accidents.zip')

# extract zipfile
with ZipFile(BytesIO(response.content)) as zf:
    # get our text file
    filename = zf.namelist()

    # open text file
    with zf.open(filename[0]) as file:
        # make it a dataframe
        accidents = pd.read_csv(file, on_bad_lines='skip', 
                                encoding='ISO-8859-1', delimiter='|')


accidents.head()

C:\Users\Jessie\AppData\Local\Temp\ipykernel_30280\2608785646.py:12: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  accidents = pd.read_csv(file, on_bad_lines='skip', encoding='ISO-8859-1', delimiter='|')


,MINE_ID,CONTROLLER_ID,CONTROLLER_NAME,OPERATOR_ID,OPERATOR_NAME,CONTRACTOR_ID,DOCUMENT_NO,SUBUNIT_CD,SUBUNIT,ACCIDENT_DT,...,DAYS_RESTRICT,DAYS_LOST,TRANS_TERM,RETURN_TO_WORK_DT,IMMED_NOTIFY_CD,IMMED_NOTIFY,INVEST_BEGIN_DT,NARRATIVE,CLOSED_DOC_NO,COAL_METAL_IND
0,100003,0041044,Lhoist Group,L13586,"Lhoist North America of Alabama, LLC",MPH,220132310045,3,"STRIP, QUARY, OPEN PIT",08/12/2013,...,2.0,0,N,08/19/2013,?,NO VALUE FOUND,NaN,Employee says he slipped off back of fuel truc...,NaN,M
1,100003,0041044,Lhoist Group,L13586,"Lhoist North America of Alabama, LLC",NaN,220241210019,30,MILL OPERATION/PREPARATION PLANT,04/22/2024,...,0.0,0,N,04/22/2024,?,NO VALUE FOUND,NaN,While blowing lime dust from the lime cooler a...,NaN,M
2,100009,M00024,Legacy Vulcan Corp (Form:Vulcan Materials Co),L16168,"Vulcan Construction Materials, LLC",NaN,220053250019,3,"STRIP, QUARY, OPEN PIT",11/13/2005,...,13.0,0,N,12/01/2005,?,NO VALUE FOUND,NaN,2 employees were attempting to load troughing ...,320053360009,M
3,100011,M11763,Imerys S A,L17074,Imerys Pigments LLC,NaN,220062000052,30,MILL OPERATION/PREPARATION PLANT,07/14/2006,...,0.0,0,N,07/15/2006,?,NO VALUE FOUND,NaN,Employee was pushing against pry bar. Bar slip...,NaN,M
4,100011,M11763,Imerys S A,0090005,Imerys Carbonates LLC,NaN,220090120006,30,MILL OPERATION/PREPARATION PLANT,01/08/2009,...,0.0,0,N,01/09/2009,?,NO VALUE FOUND,NaN,Employee tripped on c-channel at #6 silo. Fel...,NaN,M


## Geospatial 
##### Get address -> make them to lat and lon

In [ ]:
# grab zip file for addresses
response = requests.get('https://arlweb.msha.gov/OpenGovernmentData/DataSets/AddressofRecord.zip')

# extract zipfile
with ZipFile(BytesIO(response.content)) as zf:
    # get our text file
    filename = zf.namelist()

    # open text file
    with zf.open(filename[0]) as file:
        # make it a dataframe
        address = pd.read_csv(file, on_bad_lines='skip', 
                              encoding='ISO-8859-1', delimiter='|',
                              dtype={'ZIP_CD': str})

address.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90640 entries, 0 to 90639
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   MINE_ID         90640 non-null  int64  
 1   MINE_NAME       90639 non-null  object 
 2   CONTACT_TITLE   78085 non-null  object 
 3   NEAREST_TOWN    54317 non-null  object 
 4   BUSINESS_NAME   89992 non-null  object 
 5   STREET          59530 non-null  object 
 6   PO_BOX          28323 non-null  object 
 7   CITY            90054 non-null  object 
 8   STATE_ABBR      90038 non-null  object 
 9   FIPS_STATE_CD   90038 non-null  float64
 10  STATE           90038 non-null  object 
 11  ZIP_CD          90033 non-null  object 
 12  COUNTRY         90056 non-null  object 
 13  PROVINCE        16 non-null     object 
 14  POSTAL_CD       16 non-null     object 
 15  MINE_TYPE_CD    90419 non-null  object 
 16  MINE_STATUS     90640 non-null  object 
 17  MINE_STATUS_DT  90640 non-null 

In [4]:
!pip install pgeocode

In [ ]:
# from zip code -> get lon and lat
import pgeocode

# set it to us
nomi = pgeocode.Nominatim('us')

# drop rows where zip codes are missing
address = address.dropna(subset=['ZIP_CD'])

# get zip code column into a list
zipcode = address['ZIP_CD'].astype(str).tolist()

# get lat and lon for all zip codes
locations = nomi.query_postal_code(zipcode)

# put it in the address column
address['lat'] = locations['latitude'].values
address['lon'] = locations['longitude'].values

# view dataframe
address.head()

,MINE_ID,MINE_NAME,CONTACT_TITLE,NEAREST_TOWN,BUSINESS_NAME,STREET,PO_BOX,CITY,STATE_ABBR,FIPS_STATE_CD,...,COUNTRY,PROVINCE,POSTAL_CD,MINE_TYPE_CD,MINE_STATUS,MINE_STATUS_DT,PRIMARY_SIC_CD,COAL_METAL_IND,lat,lon
0,100003,O'Neal Quarry & Mill,Operations Manager,Calera,"Lhoist North America of Alabama, LLC",2885 Hwy 31 South,NaN,Calera,AL,1.0,...,USA,NaN,NaN,Surface,Active,01/22/1979,Lime,M,33.1098,-86.7560
1,100004,Brierfield Quarry,Quarry Manager,Brierfield,"Lhoist North America of Alabama, LLC",14786 Montevallo Road,NaN,Brierfield,AL,1.0,...,USA,NaN,NaN,Surface,Active,03/04/2003,"Crushed, Broken Limestone NEC",M,33.0427,-86.9517
2,100005,Birmingham Plant,NaN,NaN,Allied Lime Company,NaN,36130,BIRMINGHAM,AL,1.0,...,USA,NaN,NaN,Surface,Abandoned,08/15/1989,"Crushed, Broken Limestone NEC",M,33.5446,-86.9292
3,100006,Auburn Quarry,Safety Representative,Auburn,"Martin Marietta Materials, Inc.",245 Rockwood Rd,NaN,Tyrone,GA,13.0,...,USA,NaN,NaN,Surface,Active,09/24/1976,"Crushed, Broken Limestone NEC",M,33.4719,-84.5914
4,100008,Landmark Plant,Operations Manager,Alabaster,Cheney Lime & Cement Company,1152 Old Hwy 31,NaN,Alabaster,AL,1.0,...,USA,NaN,NaN,Surface,Active,11/14/1975,Lime,M,33.1934,-86.7944


In [55]:
# if mine id matches with the address -> add lat and lon columns
accidents = accidents.merge(address[['MINE_ID', 'lat', 'lon']],
                            on = 'MINE_ID', how='left')

accidents.head()

,MINE_ID,CONTROLLER_ID,CONTROLLER_NAME,OPERATOR_ID,OPERATOR_NAME,CONTRACTOR_ID,DOCUMENT_NO,SUBUNIT_CD,SUBUNIT,ACCIDENT_DT,...,INVEST_BEGIN_DT,NARRATIVE,CLOSED_DOC_NO,COAL_METAL_IND,lat_x,lon_x,lat_y,lon_y,lat,lon
0,1302196,M00097,Mark Schildberg,L00140,Schildberg Construction Company Inc,F466,220110960032,3,"STRIP, QUARY, OPEN PIT",03/29/2011,...,03/29/2011,Truck driver was helping excavator operator re...,320111180015,M,41.3133,-94.4408,41.3133,-94.4408,41.3133,-94.4408
1,1514324,C15455,Alliance Resource Partners LP,P24012,Excel Mining LLC,U306,220112380057,30,MILL OPERATION/PREPARATION PLANT,08/19/2011,...,08/19/2011,The trucks bed started sliding around to the l...,320113350015,C,38.2868,-81.8051,38.2868,-81.8051,38.2868,-81.8051
2,2601941,M04598,"Coeur Mining, Inc.",L08263,"Coeur Rochester, Inc.",Z1G,220171180016,3,"STRIP, QUARY, OPEN PIT",04/23/2017,...,04/23/2017,While walking between two four-foot culvert pi...,320182820027,M,40.1819,-118.4689,40.1819,-118.4689,40.1819,-118.4689
3,3302784,M06183,CRH PLC,0050668,Columbus Limestone Inc,SVT,220050030049,3,"STRIP, QUARY, OPEN PIT",12/20/2004,...,12/20/2004,HE WAS STRIPPING MATERIAL FROM HIGHWALL AND IT...,320050140018,M,39.8974,-82.4071,39.8974,-82.4071,39.8974,-82.4071
4,4800977,C15561,Arch Resources Inc,P24224,Thunder Basin Coal Company LLC,ZC7,220081720013,30,MILL OPERATION/PREPARATION PLANT,05/31/2008,...,05/31/2008,While setting conveyor tube on supports above ...,320090360009,C,43.7512,-105.4920,43.7512,-105.4920,43.7512,-105.4920


In [56]:
# create a new geo dataframe
import geopandas as gpd
accidents_gdf = gpd.GeoDataFrame(accidents,
                                 geometry=gpd.points_from_xy(accidents['lon'],
                                                             accidents['lat'])).set_crs("EPSG:4326", allow_override=True)
# accidents_gdf.plot(alpha=0.5, markersize=1)

In [57]:
import folium
from folium import Marker
# create a map of us
us_map = folium.Map(location=[37.0902, -95.7129], zoom_start=4)


# add points to the map
for idx, row in accidents_gdf.to_crs(epsg=4326).iterrows():
    Marker([row.geometry.y, row.geometry.x]).add_to(us_map)

us_map